In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [3]:
#<데이터수집>

email_list = [
              {'email title' : 'free game only today', 'spam' : True},
              {'email title' : 'cheapest flight deal', 'spam' : True},
              {'email title' : 'limited time offer only today only today', 'spam' : True},
              {'email title' : 'today meeting schedule' , 'spam' : False},
              {'email title' : 'your flight schedule attached', 'spam' : False},
              {'email title' : 'your credit card statement' , 'spam' : False}
]
df = pd.DataFrame(email_list)

In [4]:
df

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False
5,your credit card statement,False


In [5]:
#<데이터 전처리>

df['label'] = df['spam'].map({True : 1, False : 0})

In [6]:
df_x = df["email title"]
df_y = df["label"]

In [7]:
cv = CountVectorizer(binary = True)
x_traincv = cv.fit_transform(df_x)

In [8]:
encoded_input = x_traincv.toarray()
encoded_input

array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]])

1. 베르누이 NB는 바이너리 데이터만 처리 가능.
2. 텍스트 데이터를 바이너리로 처리가능.(CounterVectorize) -> 데이터에 포함된 단어들을 이진수의 벡터로 만들어줌
3. binary = True 를 남겨줌으로써 단어가 한 번 이상 들어가면 1 아니면 0 -> 3행을 보면 only today가 2번씩 들어갔지만 1로 표현됨.

In [9]:
#인코딩된 이메일 제목에 어떤 단어가 포함됐는지 확인하려면

cv.inverse_transform(encoded_input[0])

[array(['free', 'game', 'only', 'today'], dtype='<U9')]

In [10]:
#각 인덱스가 어떤 단어를 의미하는지 궁금하다면

cv.get_feature_names()

['attached',
 'card',
 'cheapest',
 'credit',
 'deal',
 'flight',
 'free',
 'game',
 'limited',
 'meeting',
 'offer',
 'only',
 'schedule',
 'statement',
 'time',
 'today',
 'your']

In [11]:
#[베르누이 분류기 모델 학습]

bnb = BernoulliNB()
y_train = df_y.astype('int')
bnb.fit(x_traincv, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

**베르누이BN는 기본적으로 스무딩을 지원함. -> 학습에 없는 단어가 등장하더라도 이상 없이 분류를 진행함**

In [17]:
##테스트 데이터 다듬기

test_email_list = [
                {'email title': 'free flight offer', 'spam': True},
                {'email title': 'hey traveler free flight deal', 'spam': True},
                {'email title': 'limited free game offer', 'spam': True},
                {'email title': 'today flight schedule', 'spam': False},
                {'email title': 'your credit card attached', 'spam': False},
                {'email title': 'free credit card offer only today', 'spam': False}
             ]

test_df = pd.DataFrame(test_email_list)
test_df['label'] = test_df['spam'].map({True:1, False:0})
test_x = test_df['email title']
test_y = test_df['label']
x_testcv = cv.transform(test_x)

In [18]:
##테스트

predictions = bnb.predict(x_testcv)

In [19]:
accuracy_score(test_y, predictions)

0.8333333333333334

array([1, 1, 1, 0, 0, 0])